GCN **CODE**

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score, recall_score, f1_score
import pandas as pd

# Define GCN model
class GCN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

# Load and preprocess data
train_data = pd.read_csv('/content/restrain.csv')
test_data = pd.read_csv('/content/restest.csv')

vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(train_data['Sentence']).toarray()
X_test = vectorizer.transform(test_data['Sentence']).toarray()

encoder = LabelEncoder()
y_train = encoder.fit_transform(train_data['Polarity'])
y_test = encoder.transform(test_data['Polarity'])

# Convert data to PyTorch tensors
X_train = torch.FloatTensor(X_train)
y_train = torch.LongTensor(y_train)
X_test = torch.FloatTensor(X_test)
y_test = torch.LongTensor(y_test)

# Define custom dataset
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Create datasets and data loaders
train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=200, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=200, shuffle=False)

# Define training function
def train(model, train_loader, optimizer, criterion, device):
    model.train()
    total_loss = 0.0
    for data, target in train_loader:
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)

# Define evaluation function
def evaluate(model, test_loader, criterion, device):
    model.eval()
    total_loss = 0.0
    correct = 0
    predictions = []
    ground_truths = []
    with torch.no_grad():
        for data, target in test_loader:
            data = data.to(device)
            target = target.to(device)
            output = model(data)
            total_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            predictions.extend(pred.cpu().numpy())
            ground_truths.extend(target.cpu().numpy())
    accuracy = correct / len(test_loader.dataset)
    avg_loss = total_loss / len(test_loader)
    return avg_loss, accuracy, predictions, ground_truths

# Define device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize model, optimizer, and loss function
input_dim = X_train.shape[1]
hidden_dim = 64
output_dim = len(encoder.classes_)
model = GCN(input_dim, hidden_dim, output_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

# Train and evaluate the model
num_epochs = 100
for epoch in range(1, num_epochs + 1):
    train_loss = train(model, train_loader, optimizer, criterion, device)
    test_loss, test_accuracy, predictions, ground_truths = evaluate(model, test_loader, criterion, device)
    print(f'Epoch {epoch}/{num_epochs}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')

    # Calculate precision, recall, and F1 score
    precision = precision_score(ground_truths, predictions, average='weighted')
    recall = recall_score(ground_truths, predictions, average='weighted')
    f1 = f1_score(ground_truths, predictions, average='weighted')
    print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}')

# Save trained model
torch.save(model.state_dict(), 'gcn_model.pth')


Epoch 1/100, Train Loss: 0.8377, Test Loss: 0.7096, Test Accuracy: 0.7230
Precision: 0.6972, Recall: 0.7230, F1 Score: 0.6799
Epoch 2/100, Train Loss: 0.5492, Test Loss: 0.7111, Test Accuracy: 0.7283
Precision: 0.7034, Recall: 0.7283, F1 Score: 0.7055
Epoch 3/100, Train Loss: 0.3730, Test Loss: 0.7797, Test Accuracy: 0.7113
Precision: 0.6851, Recall: 0.7113, F1 Score: 0.6884
Epoch 4/100, Train Loss: 0.3189, Test Loss: 0.8415, Test Accuracy: 0.7105
Precision: 0.6853, Recall: 0.7105, F1 Score: 0.6898
Epoch 5/100, Train Loss: 0.2756, Test Loss: 0.8954, Test Accuracy: 0.7113
Precision: 0.6810, Recall: 0.7113, F1 Score: 0.6872
Epoch 6/100, Train Loss: 0.2906, Test Loss: 0.9106, Test Accuracy: 0.7158
Precision: 0.6945, Recall: 0.7158, F1 Score: 0.7010
Epoch 7/100, Train Loss: 0.2531, Test Loss: 0.9058, Test Accuracy: 0.7096
Precision: 0.6936, Recall: 0.7096, F1 Score: 0.6951
Epoch 8/100, Train Loss: 0.2452, Test Loss: 0.9069, Test Accuracy: 0.7033
Precision: 0.6928, Recall: 0.7033, F1 Score: